In [63]:
import pandas as pd
import json
import requests as r
from pandas.io.json import json_normalize
import requests as r
from bs4 import BeautifulSoup
import re

# 1. Web scraping - main page of lespetitestables

In [64]:
url='https://www.lespetitestables.com/tous-nos-restos/'

In [65]:
html=r.get(url).content
soup=BeautifulSoup(html)

###  Determine what information I want to extract from this web page  and create nested list: 
- list of links
- Name of restaurants
- Categories
- Arrondissement
- Tagline
- Time of opening

In [66]:
listlink=[i.get('href') for i in soup.select('div.large-4 >a')][1:]

Nameresto=[i.text for i in soup.select('div.caption-text >p.caption-title')]

Price=[i.text for i in soup.select('div.price >p')]

cat= [i.get('class')[4:] for i in soup.select('div.container-isotope>div')]

Tagline=[i.text for i in soup.select('div.caption-text >p.caption-baseline')]

Time=[i.text for i in soup.select('div.caption-text >p.lunchtime')]

In [67]:
Nameresto=[i.text for i in soup.select('div.caption-text >p.caption-title')]

In [68]:
Price=[i.text for i in soup.select('div.price >p')]

In [69]:
cat= [i.get('class')[4:] for i in soup.select('div.container-isotope>div')]

In [70]:
Arr=re.findall('\d+\w+', ' '.join(Nameresto))
len(Arr)

101

In [71]:
Arr.pop(77)
len(Arr)

100

In [72]:
Tagline=[i.text for i in soup.select('div.caption-text >p.caption-baseline')]

In [73]:
Time=[i.text for i in soup.select('div.caption-text >p.lunchtime')]

### Create dataframe

In [74]:
colnames=['Nameresto','Tagline','Categories','Timelunch','Price','Link']
df=pd.DataFrame([Nameresto,Tagline, cat,Time,Price,listlink],index=colnames).T
df.head()

,Nameresto,Tagline,Categories,Timelunch,Price,Link
0,Colline d’Asie 18ème,Cantine d'Asie,"[nordest, fast-good, insolite, world]",Midi & Soir,"9,50€",https://www.lespetitestables.com/restaurant/co...
1,Burger & Fries 2ème,Burgers américains,"[nordouest, fast-good, world]",Midi & Soir,"7,95€",https://www.lespetitestables.com/restaurant/bu...
2,KIEZ Biergarten 18ème,Un biergarten à Paris,"[nordouest, fast-good, insolite, world]",Midi & Soir,13€,https://www.lespetitestables.com/restaurant/ki...
3,Coin-Op Table 9ème,Fusion Franco-Taiwanaise,"[nordest, fast-good, insolite, world]",Midi,12€,https://www.lespetitestables.com/restaurant/co...
4,Fulano 18ème,Une Cantine Argentine,"[nordest, fast-good, world]",Midi & Soir,12€,https://www.lespetitestables.com/restaurant/fu...


## 2. Web scraping - each page of lespetitestables
Determine what information I want to extract from each page

In [75]:
url1='https://www.lespetitestables.com/restaurant/burger-fries-paris/'

In [76]:
html1=r.get(url1).content
soup1=BeautifulSoup(html1)

In [77]:
ph = [i.get('src') for i in soup1.select('div.photoset-grid-custom >img')]
Adress_Tel=[i.text for i in soup1.select('div.wysiwyg p:nth-child(1)')][0]
tel =re.findall('(?:\+33|07|06|01)(?: \d{2,3}){4}', Adress_Tel)[0]
Hours=[i.text for i in soup1.select('div.wysiwyg p:nth-child(2)')]
#Price=[i.text for i in soup1.select('div.fll >span')][0]

### Boucle for to extract the informations in each pages : photos, adress, tel and price

In [17]:
photos = []
for i in listlink:
    url=i
    html=r.get(url).content
    soup=BeautifulSoup(html)
    ph = [j.get('src') for j in soup.select('div.photoset-grid-custom >img')][0]
    photos.append(ph)

In [78]:
adress =[]
tel=[]
for i in listlink:
    url=i
    html=r.get(url).content
    soup=BeautifulSoup(html)
    Adress_Tel=[k.text for k in soup.select('div.wysiwyg p:nth-child(1)')][0]
    Ad=Adress_Tel.split('\n')[1:3]
    adress.append(Ad[0])
    try:
        t=re.findall('(?:\+33|07|09|06|01)(?: \d{1,3}){4,5}', Adress_Tel)[0]
        tel.append(t)
    except:
        tel.append('Na')

In [79]:
#price=[]
#for i in listlink:
#    url=i
#    html=r.get(url).content
#    soup=BeautifulSoup(html)
#    p=[m.text for m in soup.select('div.fll >span')][0]
#    price.append(p)
#price

### Create complete data frame

In [80]:
colnames=['Nameresto','Arrondissement','Tagline','Timelunch','Categories','adress','Telephone','price','Link','photos']
df1=pd.DataFrame([Nameresto,Arr,Tagline,Time,cat,adress,tel,Price,listlink,photos],index=colnames).T
df1

,Nameresto,Arrondissement,Tagline,Timelunch,Categories,adress,Telephone,price,Link,photos
0,Colline d’Asie 18ème,18ème,Cantine d'Asie,Midi & Soir,"[nordest, fast-good, insolite, world]",21 rue André del Sarte,01 55 79 16 82,"9,50€",https://www.lespetitestables.com/restaurant/co...,https://www.lespetitestables.com/wp-content/up...
1,Burger & Fries 2ème,2ème,Burgers américains,Midi & Soir,"[nordouest, fast-good, world]",1 boulevard de Bonne Nouvelle,01 42 36 22 79,"7,95€",https://www.lespetitestables.com/restaurant/bu...,https://www.lespetitestables.com/wp-content/up...
2,KIEZ Biergarten 18ème,18ème,Un biergarten à Paris,Midi & Soir,"[nordouest, fast-good, insolite, world]",24 RUE VAUVENARGUES,+33 1 46 27 78 46,13€,https://www.lespetitestables.com/restaurant/ki...,https://www.lespetitestables.com/wp-content/up...
3,Coin-Op Table 9ème,9ème,Fusion Franco-Taiwanaise,Midi,"[nordest, fast-good, insolite, world]",17 rue de Maubeuge,Na,12€,https://www.lespetitestables.com/restaurant/co...,https://www.lespetitestables.com/wp-content/up...
4,Fulano 18ème,18ème,Une Cantine Argentine,Midi & Soir,"[nordest, fast-good, world]",« Cantina Argentina »,01 83 91 40 05,12€,https://www.lespetitestables.com/restaurant/fu...,https://www.lespetitestables.com/wp-content/up...
5,Iemanjá 3ème,3ème,L’unique tapiocaria Parisienne,Midi & Soir,"[nordest, fast-good, insolite, world]",12 Rue Dupetit-Thouars,06 64 92 78 93,"12,9€",https://www.lespetitestables.com/restaurant/ie...,https://www.lespetitestables.com/wp-content/up...
6,Levantine 10ème,10ème,Le houmous bar du canal !,Midi & Soir,"[nordest, fast-good, insolite, world]",36 rue Bichat,01 83 56 18 56,12€,https://www.lespetitestables.com/restaurant/le...,https://www.lespetitestables.com/wp-content/up...
7,Les bariolés de Maud 9ème,9ème,Une création unique,Midi & Soir,"[nordouest, design, fast-good, insolite]",12 rue Buffault,09 73 17 01 68,10€,https://www.lespetitestables.com/restaurant/ba...,https://www.lespetitestables.com/wp-content/up...
8,Café Rendez-Vous 1er,1er,Le déjeuner du marché,Midi & Soir,"[nordest, design, fast-good, romantique, tradi]",12 rue des Prêcheurs,01 40 13 90 60,12€,https://www.lespetitestables.com/restaurant/ca...,https://www.lespetitestables.com/wp-content/up...
9,Bouillon Pigalle 18ème,18ème,Tout est bon dans le bouillon,Midi & Soir,"[nordouest, design, tradi]",22 Boulevard de Clichy,01 42 59 69 31,12€,https://www.lespetitestables.com/restaurant/bo...,https://www.lespetitestables.com/wp-content/up...


## 3. Data cleaning

In [81]:
#Change Nameresto without Arr
list_names = df.Nameresto.str.split(' ')

In [82]:
list_new_names = []
for i in list_names:
    list_new_names.append(' '.join(i[:-1]))
list_new_names

['Colline d’Asie',
 'Burger & Fries',
 'KIEZ Biergarten',
 'Coin-Op Table',
 'Fulano',
 'Iemanjá',
 'Levantine',
 'Les bariolés de Maud',
 'Café Rendez-Vous',
 'Bouillon Pigalle',
 'Fuumi',
 'Bill The Butcher',
 'Pokawa',
 'Menekse',
 'Les Marmites Volantes',
 'ONI Coffee Shop',
 'Karaage-ya',
 'Bricktop Pizza',
 'As Rare',
 'Léandrés',
 'Braun Notes',
 'Coriandre',
 'Happa Teï',
 'Isana',
 'C’real',
 'La Bossue',
 'Rice Trotters',
 'Alma the chimney cake factory',
 'So Nat',
 'Ducky’s Asian Street Food',
 'Banoi',
 'Hokkaido',
 'Elaichi Vegetarian Street Food',
 'Cantine Le Myrha',
 'Ensuite',
 'Café Nata',
 'Cerido',
 'La Petite Cuisine de Sandrine',
 'At Homs’',
 'Cafézoïde',
 'Guo Xin',
 'Toto',
 'Le Milton',
 'Les Anges Gourmands',
 'La Rizière',
 'La Lingerie',
 'Chez Foucher',
 'Le Pain Vendôme',
 'Bül',
 'Chez Moon',
 'Gumbo Yaya',
 'Amo',
 'La Petite Crêperie',
 'Les Fondus de La Raclette',
 'Little Bakery',
 'Chaly Tov – Sur Le Pouce',
 'Sette Pizza',
 'Café Jeux associatif N

In [83]:
df1['Nameresto']=list_new_names

In [84]:
df1['price']=df1['price'].str.replace(',','.').str.replace('€','').astype('float')

In [86]:
df1['Telephone']=df1['Telephone'].str.replace(' ','')

In [87]:
df1.dtypes

Nameresto          object
Arrondissement     object
Tagline            object
Timelunch          object
Categories         object
adress             object
Telephone          object
price             float64
Link               object
photos             object
dtype: object

In [90]:
list_new_cat = []
for i in cat:
    list_new_cat.append(i[1:])
df1.Categories = list_new_cat

In [91]:
df1

,Nameresto,Arrondissement,Tagline,Timelunch,Categories,adress,Telephone,price,Link,photos
0,Colline d’Asie,18ème,Cantine d'Asie,Midi & Soir,"[fast-good, insolite, world]",21 rue André del Sarte,0155791682,9.50,https://www.lespetitestables.com/restaurant/co...,https://www.lespetitestables.com/wp-content/up...
1,Burger & Fries,2ème,Burgers américains,Midi & Soir,"[fast-good, world]",1 boulevard de Bonne Nouvelle,0142362279,7.95,https://www.lespetitestables.com/restaurant/bu...,https://www.lespetitestables.com/wp-content/up...
2,KIEZ Biergarten,18ème,Un biergarten à Paris,Midi & Soir,"[fast-good, insolite, world]",24 RUE VAUVENARGUES,+33146277846,13.00,https://www.lespetitestables.com/restaurant/ki...,https://www.lespetitestables.com/wp-content/up...
3,Coin-Op Table,9ème,Fusion Franco-Taiwanaise,Midi,"[fast-good, insolite, world]",17 rue de Maubeuge,Na,12.00,https://www.lespetitestables.com/restaurant/co...,https://www.lespetitestables.com/wp-content/up...
4,Fulano,18ème,Une Cantine Argentine,Midi & Soir,"[fast-good, world]",« Cantina Argentina »,0183914005,12.00,https://www.lespetitestables.com/restaurant/fu...,https://www.lespetitestables.com/wp-content/up...
5,Iemanjá,3ème,L’unique tapiocaria Parisienne,Midi & Soir,"[fast-good, insolite, world]",12 Rue Dupetit-Thouars,0664927893,12.90,https://www.lespetitestables.com/restaurant/ie...,https://www.lespetitestables.com/wp-content/up...
6,Levantine,10ème,Le houmous bar du canal !,Midi & Soir,"[fast-good, insolite, world]",36 rue Bichat,0183561856,12.00,https://www.lespetitestables.com/restaurant/le...,https://www.lespetitestables.com/wp-content/up...
7,Les bariolés de Maud,9ème,Une création unique,Midi & Soir,"[design, fast-good, insolite]",12 rue Buffault,0973170168,10.00,https://www.lespetitestables.com/restaurant/ba...,https://www.lespetitestables.com/wp-content/up...
8,Café Rendez-Vous,1er,Le déjeuner du marché,Midi & Soir,"[design, fast-good, romantique, tradi]",12 rue des Prêcheurs,0140139060,12.00,https://www.lespetitestables.com/restaurant/ca...,https://www.lespetitestables.com/wp-content/up...
9,Bouillon Pigalle,18ème,Tout est bon dans le bouillon,Midi & Soir,"[design, tradi]",22 Boulevard de Clichy,0142596931,12.00,https://www.lespetitestables.com/restaurant/bo...,https://www.lespetitestables.com/wp-content/up...


In [92]:
df1['Arrondissement']=df1['Arrondissement'].replace('02ème','2ème')
df1

,Nameresto,Arrondissement,Tagline,Timelunch,Categories,adress,Telephone,price,Link,photos
0,Colline d’Asie,18ème,Cantine d'Asie,Midi & Soir,"[fast-good, insolite, world]",21 rue André del Sarte,0155791682,9.50,https://www.lespetitestables.com/restaurant/co...,https://www.lespetitestables.com/wp-content/up...
1,Burger & Fries,2ème,Burgers américains,Midi & Soir,"[fast-good, world]",1 boulevard de Bonne Nouvelle,0142362279,7.95,https://www.lespetitestables.com/restaurant/bu...,https://www.lespetitestables.com/wp-content/up...
2,KIEZ Biergarten,18ème,Un biergarten à Paris,Midi & Soir,"[fast-good, insolite, world]",24 RUE VAUVENARGUES,+33146277846,13.00,https://www.lespetitestables.com/restaurant/ki...,https://www.lespetitestables.com/wp-content/up...
3,Coin-Op Table,9ème,Fusion Franco-Taiwanaise,Midi,"[fast-good, insolite, world]",17 rue de Maubeuge,Na,12.00,https://www.lespetitestables.com/restaurant/co...,https://www.lespetitestables.com/wp-content/up...
4,Fulano,18ème,Une Cantine Argentine,Midi & Soir,"[fast-good, world]",« Cantina Argentina »,0183914005,12.00,https://www.lespetitestables.com/restaurant/fu...,https://www.lespetitestables.com/wp-content/up...
5,Iemanjá,3ème,L’unique tapiocaria Parisienne,Midi & Soir,"[fast-good, insolite, world]",12 Rue Dupetit-Thouars,0664927893,12.90,https://www.lespetitestables.com/restaurant/ie...,https://www.lespetitestables.com/wp-content/up...
6,Levantine,10ème,Le houmous bar du canal !,Midi & Soir,"[fast-good, insolite, world]",36 rue Bichat,0183561856,12.00,https://www.lespetitestables.com/restaurant/le...,https://www.lespetitestables.com/wp-content/up...
7,Les bariolés de Maud,9ème,Une création unique,Midi & Soir,"[design, fast-good, insolite]",12 rue Buffault,0973170168,10.00,https://www.lespetitestables.com/restaurant/ba...,https://www.lespetitestables.com/wp-content/up...
8,Café Rendez-Vous,1er,Le déjeuner du marché,Midi & Soir,"[design, fast-good, romantique, tradi]",12 rue des Prêcheurs,0140139060,12.00,https://www.lespetitestables.com/restaurant/ca...,https://www.lespetitestables.com/wp-content/up...
9,Bouillon Pigalle,18ème,Tout est bon dans le bouillon,Midi & Soir,"[design, tradi]",22 Boulevard de Clichy,0142596931,12.00,https://www.lespetitestables.com/restaurant/bo...,https://www.lespetitestables.com/wp-content/up...


In [93]:
Arr_num = re.findall('\d+',' '.join(Arr))
df1['Arr']=Arr_num
df1['Arr']=df1['Arr'].astype(int)

In [94]:
df1['adress']=df1['adress'].replace("Marché couvert Saint-Quentin",'85 bis, boulevard de Magenta')
df1['adress']=df1['adress'].replace('Le Neuvième Ciel','67, rue de Rochechouart')

In [100]:
zipcode=[]
for i in df1.Arr:
    zipcode.append(f'750{str(i).zfill(2)}')
zipcode=[(f'750{str(i).zfill(2)}') for i in df1.Arr]
df1['zipcode']=zipcode


In [102]:
df1['Categories']=df1.Categories.astype(str)
df1['Categories']=df1['Categories'].str.strip('[]')

In [103]:
df1['Categories']=df1['Categories'].str.replace(r"'","")

### Save dataframe that I will need for the app

In [107]:
df1.columns
cols_to_drop=['Link']
df=df1.drop(cols_to_drop, axis=1)
column_order=['Nameresto', 'Arrondissement','Categories', 'Tagline', 'Timelunch', 'price', 'adress', 'zipcode',
       'Telephone', 'photos', 'Arr']
df=df[column_order]

In [108]:
df['Nameresto1']=list_new_names

In [62]:
import os
os.getcwd()
df.to_csv('Lespetitestablesv5.csv')

## Create the app - switch on the Appresto jupyternotebook

### Find the cheapest restaurant according to your arrondissement and the categories

In [96]:
df.set_index('Nameresto')

,Arrondissement,Categories,Tagline,Timelunch,price,Newcat,adress,zipcode,Telephone,photos,Arr,Nameresto1
Nameresto,,,,,,,,,,,,
Colline d’Asie,18ème,"fast-good, insolite, world",Cantine d'Asie,Midi & Soir,9.50,fast-good,21 rue André del Sarte,75018,0155791682,https://www.lespetitestables.com/wp-content/up...,18,Colline d’Asie
Burger & Fries,2ème,"fast-good, world",Burgers américains,Midi & Soir,7.95,fast-good,1 boulevard de Bonne Nouvelle,75002,0142362279,https://www.lespetitestables.com/wp-content/up...,2,Burger & Fries
KIEZ Biergarten,18ème,"fast-good, insolite, world",Un biergarten à Paris,Midi & Soir,13.00,fast-good,24 RUE VAUVENARGUES,75018,+33146277846,https://www.lespetitestables.com/wp-content/up...,18,KIEZ Biergarten
Coin-Op Table,9ème,"fast-good, insolite, world",Fusion Franco-Taiwanaise,Midi,12.00,fast-good,17 rue de Maubeuge,75009,Na,https://www.lespetitestables.com/wp-content/up...,9,Coin-Op Table
Fulano,18ème,"fast-good, world",Une Cantine Argentine,Midi & Soir,12.00,fast-good,« Cantina Argentina »,75018,0183914005,https://www.lespetitestables.com/wp-content/up...,18,Fulano
Iemanjá,3ème,"fast-good, insolite, world",L’unique tapiocaria Parisienne,Midi & Soir,12.90,fast-good,12 Rue Dupetit-Thouars,75003,0664927893,https://www.lespetitestables.com/wp-content/up...,3,Iemanjá
Levantine,10ème,"fast-good, insolite, world",Le houmous bar du canal !,Midi & Soir,12.00,fast-good,36 rue Bichat,75010,0183561856,https://www.lespetitestables.com/wp-content/up...,10,Levantine
Les bariolés de Maud,9ème,"design, fast-good, insolite",Une création unique,Midi & Soir,10.00,design,12 rue Buffault,75009,0973170168,https://www.lespetitestables.com/wp-content/up...,9,Les bariolés de Maud
Café Rendez-Vous,1er,"design, fast-good, romantique, tradi",Le déjeuner du marché,Midi & Soir,12.00,design,12 rue des Prêcheurs,75001,0140139060,https://www.lespetitestables.com/wp-content/up...,1,Café Rendez-Vous


In [104]:
arr=9
cat='design'
filtA=df[df.Arr==int(arr)].copy()
filtA

,Nameresto,Arrondissement,Categories,Tagline,Timelunch,price,Newcat,adress,zipcode,Telephone,photos,Arr,Nameresto1
3,Coin-Op Table,9ème,"fast-good, insolite, world",Fusion Franco-Taiwanaise,Midi,12.0,fast-good,17 rue de Maubeuge,75009,Na,https://www.lespetitestables.com/wp-content/up...,9,Coin-Op Table
7,Les bariolés de Maud,9ème,"design, fast-good, insolite",Une création unique,Midi & Soir,10.0,design,12 rue Buffault,75009,0973170168,https://www.lespetitestables.com/wp-content/up...,9,Les bariolés de Maud
19,Léandrés,9ème,"design, world",Le café où se sentir (colom)bien,Midi & Soir,9.5,design,78 Rue de Maubeuge,75009,0986795729,https://www.lespetitestables.com/wp-content/up...,9,Léandrés
20,Braun Notes,9ème,"design, world",Le coffee shop venu de Berlin,Midi,12.0,design,31-33 Rue de Mogador,75009,Na,https://www.lespetitestables.com/wp-content/up...,9,Braun Notes
23,Isana,9ème,world,Le meilleur de l’Amérique latine,Midi,12.0,world,"7 rue Bourdaloue,",75009,0142451872,https://www.lespetitestables.com/wp-content/up...,9,Isana
28,So Nat,9ème,"design, fast-good, world",Le temple du buddha bowl bio!,Midi,9.0,design,5 rue Bourdaloue,75009,0177189652,https://www.lespetitestables.com/wp-content/up...,9,So Nat
35,Café Nata,9ème,"insolite, world",Le micro-resto portugais,Midi,10.0,insolite,69 rue de Provence,75009,0632937102,https://www.lespetitestables.com/wp-content/up...,9,Café Nata
41,Toto,9ème,world,Un Italien meilleur qu'au pays !,Midi,8.8,world,35 Rue Godot de Mauroy,75009,0142652151,https://www.lespetitestables.com/wp-content/up...,9,Toto
42,Le Milton,9ème,design,Déjeunez dans un bar à cocktails,Midi,9.5,design,46 Rue Lamartine,75009,0145266220,https://www.lespetitestables.com/wp-content/up...,9,Le Milton
43,Les Anges Gourmands,9ème,"insolite, tradi",Le resto planqué,Midi,10.0,insolite,12 Rue de Rochechouart,75009,0148782883,https://www.lespetitestables.com/wp-content/up...,9,Les Anges Gourmands


In [108]:
filtAC=filtA.loc[(filtA.Categories.str.contains(str(cat)))].sort_values(by='price').head(3)
filtAC

,Nameresto,Arrondissement,Categories,Tagline,Timelunch,price,Newcat,adress,zipcode,Telephone,photos,Arr,Nameresto1
28,So Nat,9ème,"design, fast-good, world",Le temple du buddha bowl bio!,Midi,9.0,design,5 rue Bourdaloue,75009,0177189652,https://www.lespetitestables.com/wp-content/up...,9,So Nat
19,Léandrés,9ème,"design, world",Le café où se sentir (colom)bien,Midi & Soir,9.5,design,78 Rue de Maubeuge,75009,0986795729,https://www.lespetitestables.com/wp-content/up...,9,Léandrés
42,Le Milton,9ème,design,Déjeunez dans un bar à cocktails,Midi,9.5,design,46 Rue Lamartine,75009,0145266220,https://www.lespetitestables.com/wp-content/up...,9,Le Milton
7,Les bariolés de Maud,9ème,"design, fast-good, insolite",Une création unique,Midi & Soir,10.0,design,12 rue Buffault,75009,0973170168,https://www.lespetitestables.com/wp-content/up...,9,Les bariolés de Maud
48,Bül,9ème,"design, fast-good",Le resto anti grisaille !,Midi,10.0,design,20 avenue Trudaine,75009,0140160274,https://www.lespetitestables.com/wp-content/up...,9,Bül
